## Imports

In [129]:
import pandas as pd
import numpy as np
import cvxpy as cp

import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras import saving
#Tensor Flow
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras import backend as K

## Load Data

In [160]:
# Load Training Portfolio Data
train_series = "synthetic1000"
train_date = "2016-01-10"

# load portfolio mu's
df_train_mu = pd.read_csv(f'{train_series}\{train_date}\mu', header = None, names = ['mu'])
# load portfolio Q's
df_train_Q=pd.read_fwf(f'{train_series}\{train_date}\Q', header = None)


# Load Testing Portfolio Data
test_series = "synthetic1000"
test_date = "2016-02-21"

# load portfolio mu's
df_test_mu = pd.read_csv(f'{test_series}\{test_date}\mu', header = None, names = ['mu'])
# load portfolio Q's
df_test_Q=pd.read_fwf(f'{test_series}\{test_date}\Q', header = None)


## Constants

In [240]:
# portfolio cardinality
cardinality = 50

# training data size
n_training_portfolios = 500
# testing data size
n_testing_portfolios = 500


# ANN training epochs
EPOCH=1000
# ANN training batch size
BATCH_SIZE=1

## Solve the Maximum Portfolio Returns

In [ ]:
# def max_portfolio_return(n, mu):
#     '''
#     Params
#     n = total stocks in current portfolio plus the one stock selected by ANN
#     mu = array of mu values in current portfolio plus the mu of selected stock

#     Returns
#     Max portfolio returns of new portfolio according mvo weights
#     '''
 
#     # Create optimization variable for weights of stocks in portfolio
#     w = cp.Variable(n)

#     # Define Markowitz constraints
#     constraints2 = [sum(w) == 1, w >= 0]
#     prob_max_returns = cp.Problem(cp.Maximize(mu.T@w), constraints2)
                    
#     # Solve the optimization
#     return prob_max_returns.solve(verbose=False)

In [202]:
def max_portfolio_return(n, mu, Q, gamma = 1):
    '''
    Params
    n = total stocks in current portfolio plus the one stock selected by ANN
    mu = array of mu values in current portfolio plus the mu of selected stock

    Returns
    Max portfolio returns of new portfolio according mvo weights
    '''
 
    # Create optimization variable for weights of stocks in portfolio
    w = cp.Variable(n)

    # Define Markowitz constraints
    constraints = [sum(w) == 1, w >= 0]
    # prob_max_returns = cp.Problem(cp.Maximize(mu.T@w), constraints)
    risk_weighted_returns = cp.Problem(cp.Maximize((gamma*mu.T@w) - cp.quad_form(w,Q)), constraints)
                    
    # Solve the optimization
    return risk_weighted_returns.solve(verbose=False)

## Initialize Training and Testing Portfolios

We can initialize portfolios randomly that are less than the cardinality constraint

In [207]:
# Initialize Training Portfolios
training_portfolios = []
for n in range(n_training_portfolios):

    # Total stocks in training batch: (1 prediction stock + initial portfolio stocks)
    n_stocks = int(np.random.rand(1)[0]*cardinality + 1)
    
    # Select stocks from synthetic database to initialize prediction stock + portfolio
    stock_indices=np.random.choice((len(df_train_mu)-1), n_stocks, replace=False)

    # Store mu portfolio
    stock_mu = np.zeros(cardinality)
    for i, index_i in enumerate(stock_indices):
        stock_mu[i] = df_init_mu.loc[index_i]
    
    
    # Store Q portfolio
    stock_Q = np.zeros((cardinality, cardinality))
    for i,index_i in enumerate(stock_indices):
        for j,index_j in enumerate(stock_indices):
            stock_Q[i][j] = df_train_Q.iloc[index_i,index_j]

    temp_portfolio = [stock_mu,stock_Q]

    # build training portfolios
    training_portfolios.append(temp_portfolio)  

# Initialize Testing Portfolios
testing_portfolios = []
for n in range(n_testing_portfolios):

    # Total stocks in training batch: (1 prediction stock + initial portfolio stocks)
    n_stocks = int(np.random.rand(1)[0]*cardinality + 1)
    
    # Select stocks from synthetic database to initialize prediction stock + portfolio
    stock_indices=np.random.choice((len(df_test_mu)-1), n_stocks, replace=False)

    # Store mu portfolio
    stock_mu = np.zeros(cardinality)
    for i, index_i in enumerate(stock_indices):
        stock_mu[i] = df_init_mu.loc[index_i]
    
    
    # Store Q portfolio
    stock_Q = np.zeros((cardinality, cardinality))
    for i,index_i in enumerate(stock_indices):
        for j,index_j in enumerate(stock_indices):
            stock_Q[i][j] = df_test_Q.iloc[index_i,index_j]

    temp_portfolio = [stock_mu,stock_Q]

    # build training portfolios
    testing_portfolios.append(temp_portfolio) 

C:\Users\stefa\AppData\Local\Temp\ipykernel_84576\3004209276.py:14: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  stock_mu[i] = df_init_mu.loc[index_i]
C:\Users\stefa\AppData\Local\Temp\ipykernel_84576\3004209276.py:41: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  stock_mu[i] = df_init_mu.loc[index_i]


## ANN

In [242]:
def get_ann_model(input_shape):    
    # Deep stacked MLP with dropout
    model = keras.models.Sequential([
        keras.layers.Input(shape=input_shape),
        # keras.layers.Flatten(input_shape=input_shape),
        keras.layers.Dense(1024, kernel_initializer='normal', activation='relu',
                           kernel_regularizer= keras.regularizers.L1L2(l1=0.01, l2=0.01)),
        keras.layers.Dense(512, kernel_initializer='normal', activation='relu'),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(256, kernel_initializer='normal', activation='relu',
                           kernel_regularizer= keras.regularizers.L1L2(l1=0.01, l2=0.01)),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(128,kernel_initializer='normal', activation='relu'),
        keras.layers.Dense(64,kernel_initializer='normal', activation='relu'),
        keras.layers.Dense(32,kernel_initializer='normal', activation='relu'),
        keras.layers.Dense(1, activation='relu'),
    ])
    model.compile(
        optimizer="adam",
        loss="mse",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )
    return model

## Train ANN

Every unique initialized portfolio can be thought of as a separate problem to learn. In order for the ANN to have sufficient training exposure to learn the portolio combinations, we can train 10 different stocks for each 500 portfolios.


The maximum portfolio returns porblem is a regression problem. Given a randomly chosen stock, the ANN will take in the current portfolio mu vector, the covariance matrix, and the selected stock and try and predict the new maximum portfolio returns of the blended portfolio.

After training the ANN, we can loop through all the possible stocks, predict the new maximum portfolio returns of introducing that stock, and each iteration, select the highest performing stock until cardinality is reached.

In [245]:
# Load stock prediction synthetic database
synthetic_series = "synthetic1000"
synthetic_date = "2016-04-03"
df_init_mu = pd.read_csv(f'{synthetic_series}\{synthetic_date}\mu', header = None, names = ['mu'])


# Get Training Targets
train_targets=[]
for n, portfolio in enumerate(training_portfolios):

    # MVO Input Parameters
    n = len(portfolio[0])
    mu_vector = portfolio[0]
    Q_array = portfolio[1]

    # Call Optimizer
    target=max_portfolio_return(n, mu_vector, Q_array, gamma = 1)
    
    # Store targets
    train_targets.append(target)
train_targets=np.array(train_targets)

# Get Testing Targets
test_targets = []
for n, portfolio in enumerate(testing_portfolios):

    # MVO Input Parameters
    n = len(portfolio[0])
    mu_vector = portfolio[0]
    Q_array = portfolio[1]

    # Call Optimizer
    target=max_portfolio_return(n, mu_vector, Q_array, gamma = 1)
    
    # Store targets
    test_targets.append(target)
test_targets=np.array(test_targets)
    



# Linearize Training Data
df_train=[]
for portfolio in training_portfolios:
    input = np.append(portfolio[0],portfolio[1].flatten())
    df_train.append(input)
df_train=np.array(df_train)

# Linearize Testing Data
df_test=[]
for portfolio in testing_portfolios:
    input = np.append(portfolio[0],portfolio[1].flatten())
    df_test.append(input)
df_test=np.array(df_test)


# Train ANN
         
# Store Results
model_history=[]
model_predictions=[]

# Call Model
input_shape = df_train.shape[1]
model_ANN = get_ann_model(input_shape)

print('-----Training RNN Model-----')
rnn_history = model_ANN.fit(df_train, train_targets,  
                            epochs=EPOCH, batch_size=BATCH_SIZE) 
                            #callbacks=[lr, es,mc_rnn])      # Can put callbacks in later

-----Training RNN Model-----
Epoch 1/1000
500/500 [==============================] - 20s 37ms/step - loss: 90.1911 - root_mean_squared_error: 3.2032
Epoch 2/1000
500/500 [==============================] - 19s 39ms/step - loss: 13.6289 - root_mean_squared_error: 3.2032
Epoch 3/1000
500/500 [==============================] - 19s 38ms/step - loss: 13.6182 - root_mean_squared_error: 3.2032
Epoch 4/1000
500/500 [==============================] - 18s 36ms/step - loss: 13.6283 - root_mean_squared_error: 3.2032
Epoch 5/1000
500/500 [==============================] - 18s 37ms/step - loss: 13.6974 - root_mean_squared_error: 3.2032
Epoch 6/1000
500/500 [==============================] - 19s 38ms/step - loss: 13.7251 - root_mean_squared_error: 3.2032
Epoch 7/1000
500/500 [==============================] - 18s 37ms/step - loss: 13.7278 - root_mean_squared_error: 3.2032
Epoch 8/1000
500/500 [==============================] - 18s 36ms/step - loss: 13.7303 - root_mean_squared_error: 3.2032
Epoch 9/100

KeyboardInterrupt: 